# Custom Loss function in tensorflow

In [1]:
import tensorflow as tf
print(tf.__version__)

2.0.0-beta1


Example: Implementing Huber Loss

The mean squared error might penalize large errors too much, so your model will end up being imprecise. 
The mean absolute error would not penalize outliers as much, but training might take a while to converge and the trained model might not be very precise. 
This is probably a good time to use the Huber loss.

In [10]:
def huber_loss(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

In [21]:
# Explicitly give the data type as tf.float32 to avoid InvalidArgumentError
y_true = tf.constant([1, 1.5, 2, 3, 4, 5], dtype=tf.float32)
y_pred = tf.constant([1, 1, 1, 1, 1, 1], dtype=tf.float32)

# calculating the loss
loss = huber_loss(y_true, y_pred)

In [22]:
loss.numpy()

array([0.   , 0.125, 0.5  , 1.5  , 2.5  , 3.5  ], dtype=float32)

### Next, you can just use this loss when you compile the Keras model, then train your model:
```
model.compile(loss=huber_loss, optimizer="nadam")
model.fit(X_train, y_train, [...])
```

And that’s it! For each batch during training, tf.keras will call the huber_loss() function
to compute the loss, and use it to perform a Gradient Descent step. Moreover, it will
keep track of the total loss since the beginning of the epoch, and it will display the
mean loss.

## Saving models with custom losses

#### But what happens to this custom loss when we save the model?

Saving a model containing a custom loss function actually works fine, as Keras just
saves the name of the function. However, whenever you load it, you need to provide a
dictionary that maps the function name to the actual function. More generally, when
you load a model containing custom objects, you need to map the names to the
objects:
```
model = keras.models.load_model("my_model_with_a_custom_loss.h5",
                                custom_objects={"huber_loss": huber_loss})
```

In [23]:
# With the current implementation, any error between -1 and 1 is considered “small”.
# But what if we want a different threshold? One solution is to create a function that
# creates a configured loss function:
def create_huber(threshold=1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss = threshold * tf.abs(error) - threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn

In [37]:
# calculating the loss
loss = create_huber(2.0)(y_true, y_pred)
print("loss: ", loss.numpy())
print("mean_loss: ", tf.reduce_mean(loss).numpy())

loss:  [0.    0.125 0.5   2.    4.    6.   ]
mean_loss:  2.1041667


Then use the below to compile the model, 
```
model.compile(loss=create_huber(2.0), optimizer="nadam")
```

Unfortunately, when you save the model, the threshold will not be saved. This means
that you will have to specify the threshold value when loading the model (**note that
the name to use is "huber_loss" , which is the name of the function we gave tf.keras, not
the name of the function that created it**):
```
model = keras.models.load_model("my_model_with_a_custom_loss_threshold_2.h5",
                                custom_objects={"huber_loss": create_huber(2.0)})
```

You can solve this by creating a subclass of the keras.losses.Loss class, and imple‐
ment its get_config() method:

In [32]:
class Huberloss(tf.keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss = self.threshold * tf.abs(error) - self.threshold**2 / 2
        # Note that you have to return individual losses, 
        # tf.keras will take care of finding the cumulative loss and averaging
        return tf.where(is_small_error, squared_loss, linear_loss)
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

In [33]:
huber_loss = Huberloss(2.0)

In [34]:
# calculating the loss
loss = huber_loss(y_true, y_pred)
print(loss.numpy())

2.1041667


In [39]:
huber_loss.get_config()

{'reduction': 'auto', 'name': None, 'threshold': 2.0}

You can then use any instance of this class when you compile the model:
```
model.compile(loss=HuberLoss(2.), optimizer="nadam")
```

When you save the model, the threshold will be saved along with it, and when you 
load the model you just need to map the class name to the class itself:
```
model = keras.models.load_model("my_model_with_a_custom_loss_class.h5",
                                custom_objects={"HuberLoss": HuberLoss})
```

When you save a model, Keras calls the loss instance’s get_config() method and
saves the config as JSON in the HDF5 file. When you load the model, it calls the
from_config() class method on the HuberLoss class: this method is implemented by
the base class ( Loss ) and just creates an instance of the class, passing **config to the
constructor.